In [ ]:
%matplotlib inline

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Resize

In [ ]:
from torch.utils.data import random_split, Subset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, header=None)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        image = image.float()
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
        

In [ ]:
def train_test_split(all_data, test_portion = 0.3, seed = 0):
  training_size = int((1 - test_portion) * len(all_data))
  test_size = len(all_data) - training_size
  training_indices, test_indices = random_split(
                              range(len(all_data)), 
                              [training_size, test_size],
                              generator=torch.Generator().manual_seed(seed))
  training_data = Subset(all_data, training_indices)
  test_data = Subset(all_data, test_indices)
  return training_data, test_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Download training data from open datasets.

all_data = CustomImageDataset(
    annotations_file="/content/drive/MyDrive/Ai builders/annotations_file.csv", 
    img_dir="/content/drive/MyDrive/Ai builders/pig pics cleaned", 
    transform=Resize(size=128), 
    target_transform=None
)

training_data, test_data = train_test_split(all_data)

print("All data size: " + str(len(all_data)))
print("Training size: " + str(len(training_data)) + "\tTest size: " + str(len(test_data)))

All data size: 511
Training size: 357	Test size: 154


In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 128, 128])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
from torchvision import models
model = models.resnet18(pretrained=True)
# model = models.resnet50(pretrained=True)
model.fc =nn.Linear(512,2)

Using cpu device


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            # print(f"X.Shape: {X.shape}")
            pred = model(X)
            # print(f"pred: {pred}")
            # print(f"pred shape: {pred.shape}")
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.135049  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss: 0.674036 

Epoch 2
-------------------------------
loss: 0.749447  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss: 0.825785 

Epoch 3
-------------------------------
loss: 0.675412  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss: 0.781984 

Epoch 4
-------------------------------
loss: 0.628676  [    0/  357]
Test Error: 
 Accuracy: 68.2%, Avg loss: 0.739735 

Epoch 5
-------------------------------
loss: 0.588562  [    0/  357]
Test Error: 
 Accuracy: 68.8%, Avg loss: 0.705781 

Epoch 6
-------------------------------
loss: 0.553215  [    0/  357]
Test Error: 
 Accuracy: 70.8%, Avg loss: 0.679181 

Epoch 7
-------------------------------
loss: 0.522037  [    0/  357]
Test Error: 
 Accuracy: 71.4%, Avg loss: 0.658797 

Epoch 8
-------------------------------
loss: 0.494419  [    0/  357]
Test Error: 
 Accuracy: 72.7%, Avg loss: 0.642534 

Epoch 9
----------------

In [ ]:
torch.save(model.state_dict(), "resnet_50_model_128.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
from sklearn.metrics import classification_report
def get_incorrect_predictions(model, test_dataloader):
  out1 = []
  out2 = []
  
  for i in range(len(test_data)):
    model.eval()
    x, y = test_data[i][0], test_data[i][1]
    
    # Turn it into 4D tensor [N, C, H, W] just like in dataloader
    x = x.unsqueeze(0)
    
    with torch.no_grad():
      
      pred = model(x)
      predicted, actual = pred[0].argmax(0), y
   
      out1.append(predicted.item())
      out2.append(y)
       
  return out1, out2
y_true,y_pred = get_incorrect_predictions(model,test_dataloader)
print(classification_report(y_true,y_pred))


              precision    recall  f1-score   support

           0       0.97      0.80      0.88       128
           1       0.48      0.88      0.62        26

    accuracy                           0.82       154
   macro avg       0.73      0.84      0.75       154
weighted avg       0.89      0.82      0.84       154

